<a href="https://colab.research.google.com/github/scottwmwork/DS-Unit-2-Applied-Modeling/blob/master/module3/assignment_applied_modeling_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Applied Modeling, Module 3

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] Continue to iterate on your project: data cleaning, exploration, feature engineering, modeling.
- [ ] Make at least 1 partial dependence plot to explain your model.
- [ ] Share at least 1 visualization on Slack.

(If you have not yet completed an initial model yet for your portfolio project, then do today's assignment using your Tanzania Waterpumps model.)

## Stretch Goals
- [ ] Make multiple PDPs with 1 feature in isolation.
- [ ] Make multiple PDPs with 2 features in interaction.
- [ ] If you log-transformed your regression target, then convert your PDP back to original units. 
- [ ] Use Plotly to make a 3D PDP.
- [ ] Make PDPs with categorical feature(s). Use Ordinal Encoder, outside of a pipeline, to encode your data first. If there is a natural ordering, then take the time to encode it that way, instead of random integers. Then use the encoded data with pdpbox.I Get readable category names on your plot, instead of integer category codes.

## Links
- [Christoph Molnar: Interpretable Machine Learning — Partial Dependence Plots](https://christophm.github.io/interpretable-ml-book/pdp.html) + [animated explanation](https://twitter.com/ChristophMolnar/status/1066398522608635904)
- [Kaggle / Dan Becker: Machine Learning Explainability — Partial Dependence Plots](https://www.kaggle.com/dansbecker/partial-plots)
- [Plotly: 3D PDP example](https://plot.ly/scikit-learn/plot-partial-dependence/#partial-dependence-of-house-value-on-median-age-and-average-occupancy)

In [2]:
import sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install packages in Colab
    !pip install pdpbox
    !pip install category_encoders==2.0.0
    !pip install pandas-profiling==2.3.0
    !pip install plotly==4.1.1

     |████████████████████████████████| 57.7MB 1.3MB/s 
  Created wheel for pdpbox: filename=PDPbox-0.2.0-cp36-none-any.whl size=57690723 sha256=25c8679c926bf6e79b1b2faf0ca9c753a088755af1433dcf05e71da4f4e2ab9e
  Stored in directory: /root/.cache/pip/wheels/7d/08/51/63fd122b04a2c87d780464eeffb94867c75bd96a64d500a3fe
Successfully built pdpbox
     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 614kB 48.1MB/s 
     |████████████████████████████████| 225kB 43.8MB/s 
     |████████████████████████████████| 768kB 44.2MB/s 
     |████████████████████████████████| 194kB 51.8MB/s 
     |████████████████████████████████| 51kB 20.6MB/s 
     |████████████████████████████████| 51kB 21.8MB/s 
     |████████████████████████████████| 737kB 44.2MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-2.3.0-py2.py3-none-any.whl size=145035 sha256=66bb3ff54f704191c3d34358010476bd071cccfa5b3a7c39

In [0]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/scottwmwork/datasets/master/tmdb_5000_movies.csv')

In [4]:
df.isna().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
dtype: int64

In [0]:
#Make Genres a compatible feature

X = df #Copy df into X (Doing this so I can easily copy the following code into my wrangle function)

import ast
import numpy as np

genre = []
for x in X['genres']:
  if x == '[]':
    genre.append(np.nan)
  else:
    temp = ast.literal_eval(x) 
    genre.append(temp[0]['name']) #grabs first genre in list of dictionaries
    
#create new column for dataframe
X['genre_first_listed'] = genre

In [6]:
X['genre_first_listed'].value_counts().unique

<bound method Series.unique of Drama              1207
Comedy             1042
Action              754
Adventure           339
Horror              300
Crime               195
Thriller            194
Animation           123
Fantasy             117
Romance             106
Science Fiction      96
Documentary          89
Family               56
Mystery              41
Music                34
Western              27
History              25
War                  24
TV Movie              4
Foreign               2
Name: genre_first_listed, dtype: int64>

In [0]:
df['release_date'] = pd.to_datetime(df['release_date'],infer_datetime_format = True)
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month
df['release_day'] = df['release_date'].dt.month
df = df.drop(columns = 'release_date')

In [0]:
#Isolate the test set
test = df[df['release_year'] == 2016]
y_test = test['revenue']
X_test = test.drop(columns = 'revenue')


#Exclude test set from data
dfn = df[df['release_year'] != 2016]

In [0]:
#Create train and validation data 
from sklearn.model_selection import train_test_split
train, val = train_test_split(dfn, train_size = .80, test_size = 0.20, random_state = 42)

y_train = train.revenue
X_train = train.drop(columns = 'revenue')

y_val = val.revenue
X_val = val.drop(columns = 'revenue')

In [0]:
import ast
import numpy as np

def wrangle(X):
  
  X = X.copy()
  X = X.reset_index()

  #Make genres column usable
  genre = []
  for x in X['genres']:
    if x == '[]':
      genre.append(np.nan)
    else:
      temp = ast.literal_eval(x) 
      genre.append(temp[0]['name']) #grabs first genre in list of dictionaries
    

  #Features to not include:
  X = X.drop(columns = ['genres','homepage','keywords','overview','production_companies','production_countries','spoken_languages','tagline'])
  
  #Engineer features:
  
  #original title is same as title?    
  title_changed = []
  for x in range(0,len(X['title'])):
    if X['title'][x] == X['original_title'][x]:
       title_changed.append(0)
    else:
       title_changed.append(1)
  
  #length of title
  length_of_title = []
  for x in X['title']:
    length_of_title.append(len(x))
  
  #Add features to dataframe
  X['title_changed'] = title_changed
  X['length_of_title'] = length_of_title
  X['genre_first_listed'] = genre
   
  
  return X

In [0]:
#Wrangle data
X_test = wrangle(X_test)
X_val = wrangle(X_val)
X_train = wrangle(X_train)

In [0]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import category_encoders as ce
from sklearn.tree import DecisionTreeClassifier

pipeline = make_pipeline(
  # SimpleImputer(strategy = 'mean'),
  ce.OneHotEncoder(use_cat_names = True),
  SimpleImputer(),  
#   DecisionTreeClassifier(random_state = 5),
  LinearRegression()
#     LogisticRegression()
  
)

pipeline.fit(X_train,y_train)

In [15]:
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import category_encoders as ce
from sklearn.impute import SimpleImputer 

param_distributions = { 
    'n_estimators': randint(50, 500), 
    'max_depth': [5, 10, 15, 20, None], 
    'max_features': uniform(0, 1), 
}

search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42), 
    ce.OneHotEncoder(use_cat_names = True),
    SimpleImputer(),  
    param_distributions=param_distributions, 
    n_iter=5, 
    cv=2, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

TypeError: ignored

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt

examples = pd.concat([X_train, X_val])
for revenue in range(1000000,1000000000):
    X_train['revenue'] = revenue
    preds = search.predict(X_val)
    for pred in preds:
      plt.scatter(revenue, pred, color='grey')
    plt.scatter(revenue, np.mean(preds), color='red')
    plt.title('Partial Dependence')
    plt.xlabel('Income')
    plt.ylabel('Interest Rate')

NotFittedError: ignored